In [2]:
import tensorflow as tf
import argparse
import os.path
from cifar100_models import *
from cifar100_input import *
from utils import *

tf.logging.set_verbosity(tf.logging.INFO)

#### SET THIS BEFORE RUNNING
NAME = "cifar100/cifar100_scm_0.0_0.0_0.0_0.2"

try:
    checkpoints = sorted(["{}/{}".format(ROOT.format(NAME),x[:-5]) for x in os.listdir(ROOT.format(NAME)) if x.endswith(".meta")], 
                     key=lambda x: int(x.split("_")[-1]))
except FileNotFoundError:
    checkpoints = []


In [6]:
#### Rows are true label, columns are the prediction
def p_confusion_matrix(model, checkpoint=None):
    cf = np.zeros((100,100))
    with tf.Session() as sess:
        model._restore_or_init(sess, checkpoint)
        try:
            while True:
                probas, y_ = sess.run([model.proba, model.y_])
                index = np.argmax(y_, axis=1)
                for i in range(len(index)):
                    cf[index[i]] += probas[i]
        except tf.errors.OutOfRangeError as e:
            pass
    return cf

def cm_by_time(model, checkpoints, save_file_name, save_rate=10):
    try:
        cfs = np.load(save_file_name)
        cfs = list(cfs)
    except Exception as e:
        print(e)
        cfs = []
    for i in range(len(cfs), len(checkpoints)):
        checkpoint = checkpoints[i]
        cfs.append(p_confusion_matrix(model, checkpoint))
        if not i%save_rate:
            print(i/len(checkpoints))
            np.save(save_file_name, cfs)
    return cfs

In [8]:
import os

tf.reset_default_graph()
train_files, test_files, validation_files = maybe_download_and_extract()
train_data = Cifar100Record(train_files, epochs=1)
test_data = Cifar100Record(test_files, epochs=1)

model = Cifar100ShallowConvolutionalModel(train_data.image, train_data.label)

train_cms = cm_by_time(model, checkpoints, "cifar100_train_cm.npy", 10)

tf.reset_default_graph()
train_files, test_files, validation_files = maybe_download_and_extract()
train_data = Cifar100Record(train_files, epochs=1)
test_data = Cifar100Record(test_files, epochs=1)

model = Cifar100ShallowConvolutionalModel(test_data.image, test_data.label)

test_cms = cm_by_time(model, checkpoints, "cifar100_test_cm.npy", 10)




[Errno 2] No such file or directory: 'cifar100_train_cm.npy'
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.0_0.2/model.ckpt_0000000400
0.0
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.0_0.2/model.ckpt_0000000800
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.0_0.2/model.ckpt_0000001200
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.0_0.2/model.ckpt_0000001600
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.0_0.2/model.ckpt_0000002000
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.0_0.2/model.ckpt_0000002400
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.0_0.2/model.ckpt_0000002800
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.0_0.2/model.ckpt_0000003200
INFO:tensorflow:Restoring parameters fr

In [9]:
#### TODO: may want to use normalization as in Feinberg 1970 An iterative procedure for estimation in contingency tables
for i in range(len(test_cms)):
    for row in range(len(test_cms[i])):
        test_cms[i][row] = test_cms[i][row]/np.sum(test_cms[i][row])
        train_cms[i][row] = train_cms[i][row]/np.sum(train_cms[i][row])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [10]:
x = range(len(test_cms))
print(len(test_cms))

30


In [ ]:
import scipy.signal
import matplotlib.pyplot as plt
filt = scipy.signal.gaussian(10,3)
filt /= sum(filt)
fig, axarr = plt.subplots(100,100)
test_cms = np.array(test_cms)
train_cms = np.array(train_cms)
for r in range(100):
    for c in range(100):
        axarr[r,c].set_xticklabels([])
        axarr[r,c].set_yticklabels([])
        axarr[r,c].plot(x[10:-10], scipy.signal.convolve(test_cms[:,r,c], filt,mode='same')[10:-10],'b')
        axarr[r,c].plot(x[10:-10], scipy.signal.convolve(train_cms[:,r,c], filt,mode='same')[10:-10],'r')
fig.set_size_inches(200, 200)
#plt.show()
fig.savefig('confusion_matrix_by_time_cifar100.png')